# **Graded Challenge 7**

---

# **1. Introduction**

Name  : Frederick Kurniawan Putra

Batch : HCK-016  

**Objective**  
The Goal of this project is create ann model to analyze sentiment on content in social media to know if the content is hate speech or non hate speech.

# **2. Import Libraries**  

In [ ]:
pip install feature-engine

In [ ]:
!pip install --upgrade scikit-learn joblib

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
pip install tensorflow-addons

In [ ]:
!pip install --upgrade imbalanced-learn

In [ ]:
# For Data Manipulation
import pandas as pd
import numpy as np

# For Data Visualization
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns

# For Feature Engineering
from sklearn.model_selection import train_test_split
from feature_engine.outliers import Winsorizer
from scipy.stats import kendalltau, spearmanr, pearsonr
from sklearn.preprocessing import StandardScaler
# from sklearn.compose import ColumnTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import OrdinalEncoder

# For Model Definition & Training
# from sklearn.pipeline import Pipeline
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.svm import SVR
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor

# For Model Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, cross_val_score


# For Saving Models
import pickle

# To Deactivate Warnings
import warnings
warnings.filterwarnings('ignore')

# Displaying all columns
pd.options.display.max_columns = 200

from nltk.tokenize import word_tokenize
from wordcloud import STOPWORDS
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import spacy
import re
from wordcloud import WordCloud
import nltk

#paralel processing
import multiprocessing as mp

import sys

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, GRU, Dropout, Reshape
import tensorflow as tf
from tensorflow.keras.initializers import RandomNormal, HeNormal
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as tf_hub

from sklearn.metrics import ( precision_score, recall_score, f1_score, roc_auc_score,
                             classification_report, confusion_matrix)

import torch
import torch.nn as nn

from sklearn.utils.class_weight import compute_class_weight

import tensorflow_addons as tfa
from imblearn.under_sampling import RandomUnderSampler


**Import Libraries - Explanation**


Before we commence the project, we `import all the necessary libraries required to facilitate various tasks throughout the project`. Each library serves as a tool for specific purposes as indicated by the comments provided above the code. These libraries provide functionalities ranging from data manipulation, visualization, and model building, etc., enabling us to efficiently conduct data analysis and develop machine learning models for our project.


# **3. Data Loading**

## Import Dataset

In [ ]:
#see all columns name
pd.set_option('display.max_columns',None)

#import data from csv
df = pd.read_csv('HateSpeechDatasetTrimmer.csv') # input your file name

In [ ]:
#Checking data summary
df.info()

**Checking Data Summary - Insight**

* There are 9999 entries in dataset with no entries with null values.
* There are 3 Columns in this dataset. With Content column represents text in social media, and Label that represents Classification of said content.
* Content_int column are column that won't be used for this project and will be dropped.

In [ ]:
# Identifying Target Column
df.columns

**Import dataset - Explanation**

During data loading, first we import dataset from provided csv file and extract all data inside it. All data will be inserted into initial dataFrame `df`

## Drop Content_int column and inputted error data

In [ ]:
df.drop('Content_int',axis=1,inplace=True)

In [ ]:
df[df['Label'] == 'Label'].index.to_list()

In [ ]:
df.drop(df[df['Label'] == 'Label'].index.to_list(),inplace=True)

## Change initial data type to appropriate one

In [ ]:
df['Label'] = df['Label'].astype(int)

In [ ]:
df.info()

## Check Data Duplicates

In [ ]:
# Check Data Duplicates
df.duplicated().sum()

**Check Data Duplicates - Explanation**

The code above helps us find out if there are any duplicate entries in our dataset. When we run it, and the output shows 0, it means `there are no repeated rows or data duplicates in our dataset`. Therefore, we don't need to do anything further to handle duplicated data.

In [ ]:
duplicated_text= df[df.duplicated(subset=['Content'], keep=False)].sort_values(by='Content')
duplicated_text

In [ ]:
# splitted_sentence = df['Content'].str.split()

# temp_list = []
# temp_list_2 = []

# for sentence in splitted_sentence:
#     temp_list.append(len(sentence))

# for numeric in df['Content_int'].str.split():
#     temp_list_2.append(len(numeric))

# df_cek = pd.DataFrame({
#     "length_sentence" : temp_list,
#     "numeric" : temp_list_2
# })

# display(df_cek)



In [ ]:
print("Number of Duplicated Rows based on 'Content' column: {}"
      .format(df.duplicated(subset=['Content']).sum()))
print("Number of Duplicated Rows based on 'Content' and 'Label' columns: {}"
      .format(df.duplicated(subset=['Content', 'Label']).sum()))

In [ ]:
# Group by 'text' and calculate the number of unique values for 'target'
grouped_df_train = df.groupby(['Content'])['Label'].agg(['nunique'])

# Filter groups where the number of unique 'target' values is greater than 1
df_train_mislabeled = grouped_df_train.query("nunique > 1")

print('Total Mislabeled:', df_train_mislabeled.shape[0])
df_train_mislabeled.head()

In [ ]:
# Drop columns where both the text and target have identical values.
df.drop_duplicates(subset=['Content', 'Label'],keep='first',inplace=True)

In [ ]:
print("Number of Duplicated Rows based on 'Content' column: {}"
      .format(df.duplicated(subset=['Content']).sum()))
print("Number of Duplicated Rows based on 'Content' and 'Label' columns: {}"
      .format(df.duplicated(subset=['Content', 'Label']).sum()))

In [ ]:
duplicated_text= df[df.duplicated(subset=['Content'], keep=False)].sort_values(by='Content')
duplicated_text[['Content','Label']]

In [ ]:
#drop all entries that the content has duplicate entries that have other result label
duplicate_index = df[df.duplicated(subset=['Content'], keep=False)].index.to_list()
df.drop(duplicate_index,inplace=True)

In [ ]:
print("Number of Duplicated Rows based on 'Content' column: {}"
      .format(df.duplicated(subset=['Content']).sum()))
print("Number of Duplicated Rows based on 'Content' and 'Label' columns: {}"
      .format(df.duplicated(subset=['Content', 'Label']).sum()))

In [ ]:
# Check Data Duplicates
df.duplicated().sum()

In [ ]:
df.info()

## Checking Missing Value

In [ ]:
def count_misval(df=None, show='both'):

    # Calculate the total missing value in each column
    missing_values = df.isnull().sum()

    # Calculate the percentage of missing values in each column
    percentage_missing_values = (missing_values / len(df)) * 100

    # Create a new dataframe to store the total and percentage of missing values
    missing_df = pd.DataFrame(
        {'total missing values': missing_values,
         'percentage': percentage_missing_values.round(2)})

    #Use this option to select the conditions to be displayed
    show_options = {
        'misval': missing_df[missing_df['total missing values'] > 0],
        'non_misval': missing_df[missing_df['total missing values'] == 0],
        'both': missing_df
    }

    return show_options.get(show, "Invalid value for 'show' parameter. Use 'both', 'misval', or 'non_misval'")

In [ ]:
count_misval(df=df,show='both')

# **4. Exploratory Data Analysis**


In [ ]:
nltk.download('stopwords')

## a. Distribution of Words in Dataset

In [ ]:
# show length of stopwords list from nltk stopwords
nltk_stopwords = set(stopwords.words('english'))
print('length of stopword using nltk:',len(nltk_stopwords))

#show length of stopwords list from wordcloud
print('length of stopword using wordcloud:',len(STOPWORDS))


In [ ]:
#Convert text to lowercase
df['Content']= df['Content'].apply(lambda row: str(row).lower())

In [ ]:
def calculate_word_metrics(df, col: str = None):

    # Convert the text in the specified column to lowercase
    df[col] = df[col].apply(lambda row: str(row).lower())

    # Calculate word metrics
    word_count = df[col].apply(lambda row: len(str(row).split()))
    unique_word_count = df[col].apply(lambda row: len(set(str(row).split())))
    stop_word_count = df[col].apply(lambda row: len([w for w in str(row).split() if w in STOPWORDS]))

    df_word_metrics= pd.DataFrame({
        'word_count': word_count,
        'unique_word_count': unique_word_count,
        'stop_word_count': stop_word_count,
    })

    return df_word_metrics

In [ ]:
df_metrics= calculate_word_metrics(df, col='Content')


df_metrics.head()

In [ ]:
def additional_metrics(df, col: str = None):

    # Calculate additional text metrics
    str_punctuation= '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    char_count = df[col].apply(lambda x: len(str(x)))
    punctuation_count = df[col].apply(lambda x: len([c for c in str(x) if c in str_punctuation]))
    hashtag_count = df[col].apply(lambda x: len([c for c in str(x) if c == '#']))
    mention_count = df[col].apply(lambda x: len([c for c in str(x) if c == '@']))
    url_count = df[col].apply(lambda x: len([w for w in str(x).split() if 'http' in w or 'https' in w]))

    # Create a new DataFrame with all calculated metrics
    df_additional_metrics = pd.DataFrame({
        'char_count': char_count,
        'punctuation_count': punctuation_count,
        'hashtag_count': hashtag_count,
        'mention_count': mention_count,
        'url_count': url_count
    })

    return df_additional_metrics

In [ ]:
additional_metrics(df, col='Content').head(3)

In [ ]:
#Concatenate df_metrics with additional metrics
df_metrics= pd.concat([df_metrics, additional_metrics(df, col='Content')],
                            axis = 1,join='inner')


In [ ]:
#show result from previous concatenate
print(df_metrics.shape)

In [ ]:
#create new dataframe with contatenating original dataframe with df_metrics
df_new= pd.concat([df, df_metrics], axis = 1,join='inner')

#showing the result
print(df_new.shape)

In [ ]:
def subplot_hist(df=None, columns=None, hue=None,
                 nrows=1, ncols=2, suptitle=None, palette=None):

    nrows = len(columns) // 2 + len(columns) % 2
    fig, axes = plt.subplots(nrows, ncols, figsize=(10, 8))

    # Loop through each column and create a subplot
    for i, column in enumerate(columns):
        sns.histplot(data=df, x=column, hue=hue, bins=20,
                     kde=True, ax=axes[i//2, i%2], palette=palette)

        axes[i//2, i%2].set_title(f'{column} distribution base on target', size=10)  # Set title with column name

    # Check if the number of columns is odd
    if len(columns) % 2 == 1:
        # Remove the empty subplot in the last row if the number of columns is odd
        fig.delaxes(axes[-1, -1])

    # Add super title
    if suptitle:
        plt.suptitle(suptitle, fontsize=14)

    # Adjust layout
    plt.tight_layout()

    # Show the plots
    plt.show()

In [ ]:
#choose column that has unique value more than 1
columns_to_plot = []
for column in df_new.columns[2:]:
    if df_new[column].nunique() > 1:
        columns_to_plot.append(column)

#show subplot
subplot_hist(df_new,columns= columns_to_plot,
             hue='Label', suptitle='Distribution of Dataset')

In [ ]:
df_new[columns_to_plot].describe()

## b. Percentage of Hate Speech and Non Hate Speech

In [ ]:
def generate_ngrams(text, n_gram=1):
    # Tokenize the text, remove empty strings and stopwords
    token = [token for token in text.split(' ') if token != '' if token not in STOPWORDS]

    # Generate n-grams using a sliding window approach
    ngrams = zip(*[token[i:] for i in range(n_gram)])

    # Combine the n-grams into a list of strings
    return [' '.join(ngram) for ngram in ngrams]


In [ ]:
df_new

In [ ]:
df.info()

In [ ]:
content_label1= df_new.query("Label==1")['Content']
content_label0= df_new.query("Label==0")['Content']

In [ ]:
content_label1[0]

In [ ]:
generate_ngrams(content_label1[0], n_gram=1)

In [ ]:
def ngrams_frequencies(text_data, n_grams=1, name='unigram'):
    """
    This function creates a DataFrame containing n-grams frequencies based on the provided text data.

    Parameters:
    - text_data: Iterable of text data
    - n_grams: Size of n-grams (default is 1 for unigrams)
    - name: Name to use for the n-grams column in the DataFrame (default is 'unigram')

    Returns:
    - DataFrame: DataFrame containing n-gram frequencies
    """
    # Initialize an empty dictionary to store word frequencies
    ngrams_dict = {}

    # Iterate over each text in the provided text data
    for text in text_data:
        # Generate n-grams for each word in the text
        for ngram in generate_ngrams(text, n_grams):
            # Update the frequency count for each n-gram in the word_dict dictionary
            ngrams_dict[ngram] = ngrams_dict.get(ngram, 0) + 1

    # Convert the n-gram dictionary to a DataFrame
    df_ngrams = pd.DataFrame(list(ngrams_dict.items()), columns=[name, f'{name}_counts'])

    return df_ngrams

In [ ]:
df_hate_speech_unigrams= ngrams_frequencies(content_label1) #default n_grams=1, name='unigram'
df_not_hate_speech_unigrams= ngrams_frequencies(content_label0)

In [ ]:
df_hate_speech_unigrams=df_hate_speech_unigrams.sort_values(by='unigram_counts', ascending=False)
df_not_hate_speech_unigrams=df_not_hate_speech_unigrams.sort_values(by='unigram_counts', ascending=False)

In [ ]:
df_hate_speech_unigrams.head()

In [ ]:
def custom_barplot(ax,data,x, y, top,title,color, title_size=18, fontsize=14):
    """
    Create a bar plot on the given axes.

    Parameters:
    - ax: Axes object to create the bar plot on
    - data: DataFrame containing the data for the bar plot
    - x: Column representing the x-axis values in the DataFrame
    - y: Column representing the y-axis values in the DataFrame
    - top (int): The number of top items to be displayed in the bar plot
    - title: Title for the bar plot
    - color: Color for the bars
    - title_size (int): Font size for the title (default is 20)
    - fontsize (int): Font size for other elements like axis labels (default is 14)

    Returns:
    - None
    """
    bplot= sns.barplot(data=data.head(top),
                x=x, y=y,
                orient='h', ax=ax, color=color)

    ax.set_title(title, size=title_size)
    ax.set_ylabel('')

    # Set font size for y-axis ticks
    ax.tick_params(axis='y', labelsize=fontsize)

    for i in bplot.containers:
        bplot.bar_label(i, fontsize=fontsize+2, label_type='center',color='white')


In [ ]:
#setting of figsize & subplot
fig = plt.figure(figsize = (24,12))

gs = fig.add_gridspec(10,24)
gs.update(wspace = 0.5, hspace = 0.05)


ax1 = fig.add_subplot(gs[1:12, 0:15]) #pie chart position
ax2 = fig.add_subplot(gs[1:5,12:]) #text location
ax3 = fig.add_subplot(gs[4:12,13:18]) #bar chart
ax4 = fig.add_subplot(gs[4:12,18:23]) #bar chart

# axes list
axes = [ax1,ax2,ax3,ax4]

# setting of axes; visibility of axes and spines turn off
for ax in axes:
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(True)
    #ax.set_facecolor('#f6f5f5')

    for loc in ['left', 'right', 'top', 'bottom']:
        ax.spines[loc].set_visible(False)

ax2.axes.get_xaxis().set_visible(False)
ax2.axes.get_yaxis().set_visible(False)

# ax1 (Pie Chart)
labels = ['Hate Speech', 'Non Hate Speech']
val_counts = df_new['Label'].value_counts()
# colors = colors_1[2:4]
explode = [0.08, 0]

def autopct_format(pct):
    absolute = int(pct/100.*sum(val_counts))
    return f'{pct:.2f}%\n({absolute} {labels[0] if pct < 50 else labels[1]})'

patches, texts, autotexts = ax1.pie(val_counts,
                                    explode=explode, shadow=True,
                                    startangle=90, autopct=autopct_format,
                                    textprops={'color': "black", 'weight': 'bold', 'alpha': 1, 'size': 15},
                                    wedgeprops={"edgecolor": "black", 'linewidth': 2, 'antialiased': True},
                                    pctdistance=0.5)

# Change the color of the first text
autotexts[0].set_color('w')

# # ax2 (Text)
# ax2.text(0.01, 0.8, 'Analysis of Disaster and Non-Disaster Tweets',
#          {'family': 'Serif', 'size': '25', 'weight': 'bold', 'color': 'black'},
#          bbox=dict(boxstyle='round', facecolor='lightgrey', edgecolor='none'))

# text = '''57% of the tweets are related to disasters, whereas 43% do not pertain to disasters.
# We can observe the distribution of words for each class based on unigram analysis.
# However, this dataset is not clean yet.'''
# ax2.text(0.01, 0.45, text, {'family': 'Serif', 'size': '15', 'color': 'black'})


# ax3 (barplot)

custom_barplot(ax3, df_hate_speech_unigrams,'unigram_counts','unigram', 10,
               f'Top 10 Most Frequent Unigrams \nin Hate Content','red',
               title_size=12,fontsize=10)

# ax4 (barplot)
custom_barplot(ax4, df_not_hate_speech_unigrams,'unigram_counts','unigram', 10,
               f'Top 10 Most Frequent Unigrams \nin Non-Hate Content','green',
               title_size=12,fontsize=10)

# invert x axis
ax4.invert_xaxis()
ax4.set_ylabel('')
ax4.yaxis.tick_right()

plt.show()

**Explanation**

Here we can see that our dataset has 85.58% Non Hate Speech, while hate speech represents 14.42% data. This means that the data is imbalanced.

In [ ]:
# figsize & subplot
fig = plt.figure(figsize=(20, 12))

gs = fig.add_gridspec(12, 20)
gs.update(wspace=0.5, hspace=0.05)

ax1 = fig.add_subplot(gs[1:12, :9])  # bar chart (disaster tweets)
ax2 = fig.add_subplot(gs[1:12, 10:])  # bar chart (non-disaster tweets)

# axes list
axes = [ax1, ax2]

# setting of axes; visibility of axes and spines turn off
for ax in axes:
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(True)

    for loc in ['left', 'right', 'top', 'bottom']:
        ax.spines[loc].set_visible(False)

# Create bar plot for ax1
custom_barplot(ax1, df_hate_speech_unigrams,'unigram_counts','unigram', 20,
               f'Top 20 Most Frequent Unigrams \nin Hate Speech Contents','red')

# Create bar plot for ax2
custom_barplot(ax2, df_not_hate_speech_unigrams,'unigram_counts','unigram', 20,
               f'Top 20 Most Frequent Unigrams \nin Non Hate Speech Contents','green')

# Invert x-axis for ax2
ax2.invert_xaxis()
ax2.set_ylabel('')
ax2.yaxis.tick_right()

# Add a common title for the entire subplot
fig.suptitle("Comparison of Unigrams in Hate Speech and Non Hate Speech Content", size=23)

plt.show()

**Explanation**

In this data we can see that Most frequent Unigrams that might represent mockery and racism are white, black, slut, afro, faggot. This indicates that racism slur is a indication of hate speech.

## c. Top Bigram in Hate Speech and Non Hate Speech

In [ ]:
# Calculate bigram frequencies for disaster and non-disaster tweets
df_hate_speech_bigrams= ngrams_frequencies(content_label1,n_grams=2, name='bigram')
df_non_hate_speech_bigrams= ngrams_frequencies(content_label0, n_grams=2, name='bigram')

# Sort the DataFrames based on bigram counts
df_hate_speech_bigrams=df_hate_speech_bigrams.sort_values(by='bigram_counts', ascending=False)
df_non_hate_speech_bigrams=df_non_hate_speech_bigrams.sort_values(by='bigram_counts', ascending=False)

In [ ]:
# figsize & subplot
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 12),
                         gridspec_kw={'wspace': 0.5, 'hspace': 0.05})

# axes list
ax1, ax2 = axes

# setting of axes; visibility of axes and spines turn off
for ax in axes:
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(True)

    for loc in ['left', 'right', 'top', 'bottom']:
        ax.spines[loc].set_visible(False)

# Create bar plot for ax1
custom_barplot(ax1, df_hate_speech_bigrams, 'bigram_counts', 'bigram', 20,
               'Top 20 Most Frequent Bigrams in Hate Speech Content', 'red')

# Create bar plot for ax2
custom_barplot(ax2, df_non_hate_speech_bigrams, 'bigram_counts', 'bigram', 20,
               'Top 20 Most Frequent Bigrams in Non-Hate Speech Content', 'green')

# Invert x-axis for ax2
ax2.invert_xaxis()
ax2.set_ylabel('')
ax2.yaxis.tick_right()

# Add a common title for the entire subplot
fig.suptitle("Comparison of Bigrams in Hate Speech and Non-Hate Speech Tweets", size=23)

plt.show()

**Explanation**

Here we can see that Most Frequent Bigrams in hate speech content also dominated with racial slur such as afro american, ching chong, non whites, while also followed by mockery such as shithole countries and fucking retard.

## d. Top Country Topic in Hate Speech

In [ ]:
# Loads spaCy models for NER and POS (Part-of-Speech)
language_model = spacy.load('en_core_web_sm')

class TextProcessor:
    def __init__(self, model_name="en_core_web_sm"):
        self.language_model = spacy.load(model_name)

    def apply_ner(self, text):
        doc = self.language_model(text)
        entities = [ent.text for ent in doc.ents if ent.label_ in ['PERSON']]
        return ' '.join(entities)

    def apply_pos(self, text):
        doc = self.language_model(text)
        pos_filtered = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB']]
        return ' '.join(pos_filtered)

    def clean_text_from_entities(self, text):
        doc = self.language_model(text)
        entities = [ent.text for ent in doc.ents]

        # Menghapus entitas dari teks
        for entity in entities:
            text = text.replace(entity, '')
        return text.strip()

In [ ]:
%%time
from defs import process_chunk

# Function to process each chunk of the DataFrame


# Divide the DataFrame into several chunks
# The number of chunks is equal to the number of CPU cores
total_cpu = mp.cpu_count()
chunk_size = len(df_new) // total_cpu
print(f'{chunk_size=}')
print(f'Total CPU cores: {total_cpu}')

chunks = [df_new.iloc[i:i + chunk_size]
          for i in range(0, len(df_new), chunk_size)]

#  Process each chunk in parallel using multiprocessing
with mp.Pool(mp.cpu_count()) as pool:
    processed_chunks = pool.map(process_chunk, chunks)

# Concatenate the processed chunks to get the final DataFrame
df_new = pd.concat(processed_chunks)

In [ ]:
df_new.query("Label==1").iloc[:,-2:].tail()

In [ ]:
df_new['ner_text'] = df_new['ner_text'].replace('', 'unknown')

In [ ]:
hate_speech_target_location= df_new.query("Label==1")
hate_speech_target_location.ner_text.value_counts().head()

In [ ]:
hate_speech_target_location = hate_speech_target_location.query("ner_text != 'unknown'")
hate_speech_target_location.ner_text.value_counts()

In [ ]:
combined_text = ' '.join(hate_speech_target_location['ner_text'])

# Membuat objek WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(combined_text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

**Explanation**

Here we can see that country that become a topic or target for hate speeech are africa, america, europt or london. We can safely assume based on bigram analysis that the context is refering to african american.

In [ ]:
# Split values in the 'ner_text' column & remove duplicate entities.
df_new['ner_text'] = df_new['ner_text'].apply(lambda x: ' '.join(set(x.split())))

In [ ]:
hate_speech_target_location = df_new.query("ner_text != 'unknown' and Label==1")
top10_target = hate_speech_target_location.ner_text.value_counts().head(10)
top10_target

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(True)

for loc in ['left', 'right', 'top', 'bottom']:
    ax.spines[loc].set_visible(False)

# Use the custom_barplot function
custom_barplot(ax, data=top10_target.reset_index(), x='count', y='ner_text', top=10,
               title='Top 10 Hate Speech Target',color='red')

# Show the plot
plt.show()

# 5. Feature Engineering

## Data Preprocessing

In [ ]:
def remove_stopword(text, words_to_remove=None):
    # Add words_to_remove to STOPWORDS
    if words_to_remove:
        STOPWORDS.update(words_to_remove)

    # Tokenize the text by splitting it. Remove empty strings and stopwords
    tokens = [token for token in re.split(r'\s+', text) if token != '' if token not in STOPWORDS]

    # Join tokens back into text
    processed_text = ' '.join(tokens)

    return processed_text

In [ ]:
def apply_stemming(text):
    """
    This function applies stemming to the input text using Porter Stemmer.

    Parameters:
    - text: Input text

    Returns:
    - stemmed_text: Text after stemming
    """
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in re.split(r'\s+', text) if token != '']
    stemmed_text = ' '.join(tokens)
    return stemmed_text

In [ ]:
def clean_text(text, words_to_remove=None, use_stemmer=False):
    """
    This function cleans text by removing special characters, converting text to lowercase,
    and performing additional cleaning steps as needed.

    Parameters:
    - text: Text to be cleaned
    - words_to_remove: List of words to be removed, e.g., ['will', 'one']
    - use_stemmer: Boolean, whether to use stemming or not

    Returns:
    - cleaned_text: The text after being cleaned
    """

    text = text.lower() # Convert text to lowercase
    text = remove_stopword(text,words_to_remove)#remove stopword
    text = re.sub(r'&\w+;', ' ', text) # Remove HTML entities like &amp;
    text = re.sub(r'@\S+', '', text) # Remove mentions (Twitter handles) starting with @
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text) # Remove URLs
    text = re.sub(r'\d', '', text) # Remove digits (numeric characters)

    # Remove non-alphanumeric characters (except spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Apply stemming if specified
    if use_stemmer:
        text= apply_stemming(text)

    # Replace multiple spaces with a single space and strip leading/trailing spaces
    cleaned_text = re.sub(r'\s+', ' ', text).strip()

    return cleaned_text


def clean_text_df(df, col_name, words_to_remove=None, use_stemmer=False):
    """
    This function cleans the text column in a dataframe using the clean_text function.

    Parameters:
    - df: DataFrame to be cleaned
    - col_name: Name of the text column in the dataframe
    - words_to_remove: List of words to be removed, e.g., ['will', 'one', 'body']
    - use_stemmer: Boolean, whether to use stemming or not

    Returns:
    - DataFrame with the cleaned text column
    """
    df_cleaned = df.copy()
    df_cleaned[col_name] = df_cleaned[col_name].apply(lambda x: clean_text(x, words_to_remove, use_stemmer))
    return df_cleaned


### Text Cleaning

In [ ]:
sys.setrecursionlimit(6000)

In [ ]:
df_cleaned = clean_text_df(df, 'Content', use_stemmer=True)

**Explanation**

Here we clean sentence from stopwords, mention, and html that might influence the performance of the model.

In [ ]:
calculate_word_metrics(df_cleaned, col='Content').describe()

In [ ]:
additional_metrics(df_cleaned, col='Content').describe()

In [ ]:
# Extract text data for disaster and non-disaster tweets
text_target1= df_cleaned.query("Label==1")['Content']
text_target0= df_cleaned.query("Label==0")['Content']

# Calculate bigram frequencies for disaster and non-disaster tweets
df_hate_speech_target_bigrams= ngrams_frequencies(text_target1,n_grams=2, name='bigram')
df_non_hate_speech_target_bigrams= ngrams_frequencies(text_target0, n_grams=2, name='bigram')

# Sort the DataFrames based on bigram counts
df_hate_speech_target_bigrams=df_hate_speech_target_bigrams.sort_values(by='bigram_counts', ascending=False)
df_non_hate_speech_target_bigrams=df_non_hate_speech_target_bigrams.sort_values(by='bigram_counts', ascending=False)

In [ ]:
# figsize & subplot
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 12),
                         gridspec_kw={'wspace': 0.5, 'hspace': 0.05})

# axes list
ax1, ax2 = axes

# setting of axes; visibility of axes and spines turn off
for ax in axes:
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(True)

    for loc in ['left', 'right', 'top', 'bottom']:
        ax.spines[loc].set_visible(False)

# Create bar plot for ax1
custom_barplot(ax1, df_hate_speech_target_bigrams, 'bigram_counts', 'bigram', 20,
               'Top 20 Most Frequent Bigrams in Hate Speech Content', 'red')

# Create bar plot for ax2
custom_barplot(ax2, df_non_hate_speech_target_bigrams, 'bigram_counts', 'bigram', 20,
               'Top 20 Most Frequent Bigrams in Non Hate Speech Content', 'green')

# Invert x-axis for ax2
ax2.invert_xaxis()
ax2.set_ylabel('')
ax2.yaxis.tick_right()

# Add a common title for the entire subplot
fig.suptitle("Comparison of Bigrams in Hate Speech and Non Hate Speech Content", size=23)

plt.show()

## Data Splitting

In [ ]:
df_cleaned.info()

In [ ]:
df_cleaned['Content'].info()

In [ ]:
# Data Splitting

X_train_val, X_test, y_train_val, y_test = train_test_split(df_cleaned.Content,
                                                    df_cleaned.Label,
                                                    test_size=0.15,
                                                    random_state=20,
                                                    stratify=df_cleaned.Label)

X_train, X_val, y_train, y_val = train_test_split(X_train_val,
                                                  y_train_val,
                                                  test_size=0.10,
                                                  random_state=20,
                                                  stratify=y_train_val)

print('Train Size : ', X_train.shape)
print('Val Size   : ', X_val.shape)
print('Test Size  : ', X_test.shape)

# 6. ANN Training (Sequential API/Functional API)

In [ ]:
# Tokenization using CountVectorizer
count_vec = CountVectorizer()
X_train_vec = count_vec.fit_transform(X_train)
X_val_vec = count_vec.transform(X_val)
X_test_vec = count_vec.transform(X_test)


# # #If we want to use tf.dataset, we need to convert it into an array.
# X_train_vec= X_train_vec.toarray()
# X_val_vec= X_val_vec.toarray()
# X_test_vec= X_test_vec.toarray()

In [ ]:
total_vocab = len(count_vec.vocabulary_.keys())
max_sen_len = max([len(i.split(" ")) for i in X_train])
# total_samples= len(X_train_vec)

print('Total Vocab : ', total_vocab)
print('Maximum Sentence Length : ', max_sen_len, 'tokens')
# print('Total Samples (rows):', total_samples)

In [ ]:
# Text Vectorization



text_vectorization = TextVectorization(max_tokens=20000,
                                       standardize="lower_and_strip_punctuation",
                                       split="whitespace",
                                       ngrams=None,
                                       output_mode="int",
                                       output_sequence_length=max_sen_len,
                                       input_shape=(1,)) # Only use in Sequential API

text_vectorization.adapt(X_train)

In [ ]:
# Example Result

## Document example
print('Document example')
print(df_cleaned.Content[0])
print('')

## Result of Text Vectorization
print('Result of Text Vectorization')
print(text_vectorization([df_cleaned.Content[0]]))
print('Vector size : ', text_vectorization([df_cleaned.Content[0]]).shape)

In [ ]:
# View the Top 20 Tokens (Sorted by the Highest Frequency of Appearance)

text_vectorization.get_vocabulary()[:20]

In [ ]:
# Embedding


embedding = Embedding(input_dim=20000,
                      output_dim=128,
                      embeddings_initializer="uniform",
                      input_length=max_sen_len)

In [ ]:
#Example Result

## Document example
print('Document example')
print(df_cleaned.Content[0])
print('')

## Result of Text Vectorization
print('Result of  Text Vectorization')
print(text_vectorization([df_cleaned.Content[0]]))
print('Vector size : ', text_vectorization([df_cleaned.Content[0]]).shape)
print('')

## Result of  Embedding
print('Result of  Embedding')
print(embedding(text_vectorization([df_cleaned.Content[0]])))
print('Vector size : ', embedding(text_vectorization([df_cleaned.Content[0]])).shape)

In [ ]:
def create_tf_datasets(X, y, batch_size=32, buffer_size=None, cache=False):
    # Convert to tf.data.Dataset
    ds = tf.data.Dataset.from_tensor_slices((X, y))

    AUTOTUNE = tf.data.AUTOTUNE

    # Apply prefetching to dataset
    if buffer_size is not None:
        ds = ds.shuffle(buffer_size=buffer_size, seed=3)

    ds = ds.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

    # Apply caching to the dataset if cache is set to True
    if cache:
        ds = ds.cache()

    return ds

`buffer_size` in TensorFlow, particularly in operations like `tf.data.Dataset.shuffle(buffer_size)`, refers to the number of elements from a dataset that are randomly sampled and placed in a buffer. This ensures randomness when forming batches or reading data.

In [ ]:
# buffer_size= total_samples
# train_dataset = create_tf_datasets(X_train_vec, y_train,
#                                    batch_size=32, buffer_size=None)
# val_dataset = create_tf_datasets(X_val_vec, y_val,
#                                    batch_size=32, buffer_size=None)
# test_dataset = create_tf_datasets(X_test_vec, y_test,
#                                   batch_size=32, buffer_size=None)

## 6.1. Model Definition

In [ ]:
y_train_ohe = to_categorical(y_train)
y_val_ohe = to_categorical(y_val)
y_test_ohe = to_categorical(y_test)
y_train_ohe

In [ ]:
# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),  # Classes are already 0 and 1
    y=y_train
)

# Create a dictionary mapping integer labels to their weights
class_weights_dict = dict(enumerate(class_weights))

# Create sample weights for training data
sample_weights = np.array([class_weights_dict[label] for label in y_train])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, GRU, Dropout, Reshape

## Clear Session
seed = 20
tf.keras.backend.clear_session()
np.random.seed(seed)
tf.random.set_seed(seed)

## Define the architecture
model_lstm_1 = Sequential()
model_lstm_1.add(text_vectorization)
model_lstm_1.add(embedding)
model_lstm_1.add(LSTM(32, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotUniform(seed)))
model_lstm_1.add(Dropout(0.1))
model_lstm_1.add(LSTM(16, kernel_initializer=tf.keras.initializers.GlorotUniform(seed)))
model_lstm_1.add(Dropout(0.1))
model_lstm_1.add(Dense(1, activation='sigmoid'))

model_lstm_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



## 6.2. Model Training

In [ ]:
model_lstm_1_hist = model_lstm_1.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val), batch_size=128, sample_weight=sample_weights)

## 6.3. Model Evaluation

In [ ]:
# Plot Training Results

model_lstm_1_hist_df = pd.DataFrame(model_lstm_1_hist.history)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.lineplot(data=model_lstm_1_hist_df[['accuracy', 'val_accuracy']])
plt.grid()
plt.title('Accuracy vs Val-Accuracy')

plt.subplot(1, 2, 2)
sns.lineplot(data=model_lstm_1_hist_df[['loss', 'val_loss']])
plt.grid()
plt.title('Loss vs Val-Loss')
plt.show()

In [ ]:
# Defining a function to make predictions and evaluate the model
def predict_and_evaluate(model, X,y, threshold=0.5,
                         target_names: list = None):

    y_true = []  # True labels
    y_pred = []  # Predicted labels
    y_pred_proba= []

    # Iterating through the dataset to get true labels and predictions
    predict_proba = model.predict(X, verbose=0)

    # Using tf.where to determine labels based on the threshold
    predictions = tf.where(predict_proba >= threshold, 1, 0)

    y_pred_proba.extend(predict_proba)
    y_pred.extend(predictions.numpy().flatten())

    # Iterating through the dataset to get true labels and predictions

    y_true.extend(y.array)


    y_pred_proba = np.array(y_pred_proba)
    y_pred_proba = y_pred_proba.flatten()
    cr = classification_report(y_true, y_pred, target_names=target_names)
    return cr,y_true, y_pred, y_pred_proba

In [ ]:
cr_result, y_true, y_pred,y_pred_proba = predict_and_evaluate(model_lstm_1, X_test, y_test,
                                                              target_names=['Non-Hate', 'Hate Speech'])
print(cr_result)

# 7. ANN Improvement (Sequential API/Functional API)

In [ ]:
callbacks_1 = [
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
    ModelCheckpoint(filepath='model_weight1.weights.h5', monitor='val_loss',mode='min',
                    save_weights_only=True, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)
]

## 7.1. Model Definition

### LSTM with Bidirectional LSTM and Other Tuning

In [ ]:
# Model Training using LSTM with Transfer Learning
# %%time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, GRU, Dropout, Reshape

## Clear Session
seed = 20
tf.keras.backend.clear_session()
np.random.seed(seed)
tf.random.set_seed(seed)

## Define the architecture
model_lstm_3 = Sequential()
model_lstm_3.add(text_vectorization)
model_lstm_3.add(embedding)
model_lstm_3.add(Bidirectional(LSTM(64, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotUniform(seed))))
model_lstm_3.add(Dropout(0.01))
model_lstm_3.add(Bidirectional(LSTM(32, kernel_initializer=tf.keras.initializers.GlorotUniform(seed))))
model_lstm_3.add(Dropout(0.1))
model_lstm_3.add(Dense(1, activation='sigmoid'))

model_lstm_3.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001), metrics='accuracy') #tf.keras.metrics.Precision(name='precision')



## 7.2. Model Training

In [ ]:
model_lstm_3_hist = model_lstm_3.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), batch_size=32, callbacks=callbacks_1, sample_weight=sample_weights)

## 7.3. Model Evaluation

In [ ]:
# Plot Training Results

model_lstm_3_hist_df = pd.DataFrame(model_lstm_3_hist.history)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.lineplot(data=model_lstm_3_hist_df[['accuracy', 'val_accuracy']])
plt.grid()
plt.title('Recall vs Val-Recall')

plt.subplot(1, 2, 2)
sns.lineplot(data=model_lstm_3_hist_df[['loss', 'val_loss']])
plt.grid()
plt.title('Loss vs Val-Loss')
plt.show()

In [ ]:
cr_result, y_true, y_pred,y_pred_proba = predict_and_evaluate(model_lstm_3, X_test, y_test,
                                                              target_names=['Non-Hate', 'Hate Speech'])
print(cr_result)

In [ ]:
test_loss, test_accuracy = model_lstm_3.evaluate(X_test,y_test)

# 8. Model Saving

In [ ]:
#save model
model_lstm_3.save('model_lstm_3.keras')

# 9. Model Inference

# 10. Pengambilan Kesimpulan

## Model Analysis

**Base Model Metrics**

In [ ]:
cr_result, y_true, y_pred,y_pred_proba = predict_and_evaluate(model_lstm_1, X_test, y_test,
                                                              target_names=['Non-Hate', 'Hate Speech'])
print(cr_result)

**Base Model Performance - Evaluation**

From this report, we can see that:

* On predicting Non-hate data, we can see that it has good precision, recall, and f1-score. Implying that the model can predict Non-Hate Speech Social Media Content with ease.
* However, we can see that the base model has good accuracy because it can only predict Non-hate Speech, but it doesn't have prediction performance based based on 0 score on precision, recall, anf f1-score of Hate Speech prediction score.
* Factor that might influence this low performance is because the imbalance in this dataset.

**Improved Model Metrics**

In [ ]:
cr_result, y_true, y_pred,y_pred_proba = predict_and_evaluate(model_lstm_3, X_test, y_test,
                                                              target_names=['Non-Hate', 'Hate Speech'])
print(cr_result)

**Improved Model Performance - Evaluation**

From this report, we can see that:

* The precision score on non-hate speech prediction is increased, while the recall and f1-score is is slightly decreased.

* On the other hand the Hate Speech prediction precision, recall, and f1-score increase, with the cost of decreased accuracy.

* This implies that tuning on this model increase this model Hate Speech predicting performance. Unfortunately, the performance is still not high enough to accurately predict hate speech.

**Advantages And Disadvantages of using Improved Model**

**Advantages**
* **Improved Performance than Base Model**: Tuning the base ANN model improve the performance issue on Hate-speech prediction due to imbalance dataset.

**Disadvantages**
* **Subpar Performance Overall**: Predicting performance for Hate-Speech in overall is still subpar on precision, recall, and f1-score. This implies that the model can't be used as it will tend to mispredict.

**Model Analysis Conclusion**

* That Improved model has improved performance compared to base model in terms of prediction performance on Hate-Speech Content in social media.

* However, the overall performance for improved model still results in subpar performance in correctly predicting Hate-speech Content. Using this model might giving the risk of misprediction on sentiment that will increase the workload or render the prediction result useless for project that will use this model.

## Business Insight & Further Experiments Recommendation

**Business Insights**

* Sentiment analysis on hate speech content in social media might be good to analyze public view on some instances or person such as country, people, or organization. It is necessary to look out for any harsh word or mockery that might indicate hate speech.

* Some mockery and terms that tends to racism are quite high in this dataset. This represents that social media company needs to look our for this racism term to further determine whether the some post is hate speech or not. Other company from other industries also need to watch out not to post or release any product that are having risk to be called out as racism, as it might increase the hate speech on the companie's social media post, hurting the reputation.

**Further Experiments Recommendation**

* **Balancing dataset**: Undersampling or finding other dataset that are more balanced will increase the prediction performance ot the model.
* **Relabeling**: We found that some contents in this dataset are mislabeled. It is better to reevaluate all label for content in this dataset to provide accurate dataset to be trained to model.
* **Increase Computational Power**: On this project, we only have some handful computational power, which increase the time cost for training model. By increasing the computational power, it is possible to train model with more data to increase it's accuracy and performance.
* **Explore tuning possibility**: Due to time constraint, we can't explore further of many tuning possibility. But it is possible that tuning more might also increase the score from classification report.




## Overall Analysis

This analysis shows that Improved model are a better model than based model that we've tested. After we tune the base model, we can see some improvement on Hate speech prediction. However the overall performance of this model cannot be implemented to project such as hate speech censoring or sentiment analysis for product or project for its overall subpar performance. And so it is better to further experiment on this project buy evaluating the dataset on mislabeling error and imbalanced data phenomenon. It is highly advisable to increase the computational cost to decrease time cost or possibility to train the model with bigger dataset to improve its prediction performance. Also by experimenting more on model tuning, it will also increase the performance of model to satisfaction level, which will accurately predict sentiment from the social media content whether it is a hate speech or not.